In [315]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [316]:
urls= {'Today':'http://blackdogandmagpie.net/weather/today.htm', 
       'Yesterday':'http://blackdogandmagpie.net/weather/yesterday.htm'}
lst_local = []

Part 1: Extract html tables and combine into a single list: lst_local

In [317]:
for date, url in urls.items():
    html = urlopen(url)
    soup = BeautifulSoup(html, "lxml")
    table = soup.find_all('table')[0]
    lst_table = pd.read_html(str(table))
    df_local = lst_table[0]
    df_local.columns = ['Name', 'Value', 'Time']
    df_local['Day'] = date
    lst_local.append(df_local)

Part 2: Convert the lst_local to a single DataFrame and strip down to 
        essential features needed for reporting: df_combined

In [318]:
df_combined = pd.concat(lst_local, axis=0)
df_combined['Name'] = df_combined['Name'].str.replace(u'\\xa0', u' ')
df_combined = df_combined.loc[df_combined['Name'].isin(['High Temperature', 'Low Temperature', 'High Humidity', 'Low Humidity']), ['Name', 'Day', 'Value']]
df_combined['Name'] = df_combined["Name"].astype("category")
df_combined['Name'].cat.set_categories(['High Temperature', 'Low Temperature', 'High Humidity', 'Low Humidity'],inplace=True)
# df_combined

Part 3: Pivot and relabel per reporting requirements: df_reporting

In [319]:
# df_combined.pivot_table(index='Day', columns='Name', values='Value', aggfunc=max)

In [320]:
df_reporting = df_combined.pivot_table(index='Day', columns='Name', values='Value', aggfunc=max)
df_reporting.columns = ['high_temp', 'low_temp', 'high_humidity', 'low_humidity']
df_reporting.index = ['today', 'yesterday']
df_reporting.index.name = 'day'
# df_reporting

In [321]:
df_reporting.to_csv('TakeHome_Alvin.csv')

In [322]:
df_reporting

,high_temp,low_temp,high_humidity,low_humidity
day,,,,
today,59.7 °F,40.4 °F,100 %,75 %
yesterday,65.7 °F,47.4 °F,90 %,55 %
